# **Pan-sharpening**

เทคนิค Pan-sharpening เป็นเทคนิคที่ใช้กับภาพถ่ายดาวเทียม ที่มี Panchromatic band เช่น THEOS-1, LANDSAT-8, LANDSAT-9 เพื่อให้มีรายละเอียดและมีประโยชน์มากขึ้น โดย Lab นี้เราจะทดลองกับข้อมูลดาวเทียม Landsat-9

ข้อมูลถ่ายที่เป็น Multi-Spectral มักจะมีรายละเอียดภาพไม่ค่อยดีนัก เมื่อเทียบกับข้อมูล Panchromatics  ดังนั้น การปรับความคมชัดของภาพช่วยได้โดยการรวมภาพเหล่านี้เข้ากับภาพขาวดำพิเศษที่มีรายละเอียดมากขึ้น ภาพขาวดำที่เรียกว่าภาพแพนโครมาติก มีรายละเอียดในระดับที่สูงกว่า แต่ไม่มีสี เราต้องการใช้รายละเอียดจากภาพแพนโครมาติกและสีจากภาพ Landsat-9 เพื่อสร้างภาพสีที่มีรายละเอียดสูงใหม่

ในการดำเนินการใน Lab นี้ นักศึกษา จำเป็นต้องตรวจสอบให้แน่ใจว่าข้อมูลทั้ง Panchromatic และ Multispectral ของ Landsat-9 อยู่ในตำแหน่งที่สมบูรณ์แบบ เพื่อให้เข้ากันได้อย่างลงตัว

มีหลายวิธีในการรวมภาพเหล่านี้ เช่น Brovey Transform, IHS Transform และ PCA วิธีการเหล่านี้ทำให้แน่ใจว่าภาพใหม่จะคงสีจาก Landsat-9 แต่ยังได้รับรายละเอียดเพิ่มเติมจากภาพแพนโครมาติกด้วย


เมื่อเราทำ Pan-sharpening เสร็จแล้ว เราก็จะได้ภาพใหม่ที่เป็นภาพสีที่มีรายละเอียดสูงขึ้น  โดยมีสีทั้งหมดจาก Landsat-9 และความคมชัดพิเศษจากภาพแบบแพนโครมาติก รูปภาพใหม่นี้สามารถนำไปใช้ได้หลายอย่าง เช่น ศึกษาพื้นดิน ค้นหาการเปลี่ยนแปลง หรือเพียงแค่ดูรายละเอียดเพิ่มเติมเกี่ยวกับโลก ดังนั้น การปรับความคมชัดของภาพเป็นวิธีหนึ่งในการทำให้ภาพจากดาวเทียมมีประโยชน์มากขึ้นสำหรับงานสำคัญทุกประเภท

In [1]:
# ทำการ Authenticate และ initialize Earth Engine
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-natthanichap') #อย่าลืมเปลี่ยนชื่อโปรเจคของตัวเอง

In [2]:
# กำหนดพื้นที่สนใจ (ลองเปลี่ยนเป็น Shanghai Disneyland)
geometry = ee.Geometry.Point([121.6570, 31.1434])

# เรียกภาพ Landsat 9 ตัวอย่าง แล้วดึง RGB และ Pan
image = (ee.ImageCollection("LANDSAT/LC09/C02/T1_TOA")
         .filterDate('2022-01-01', '2022-03-30')
         .filterBounds(geometry)
         .sort('CLOUD_COVER')
         .first())

In [3]:
# ทำการ Pan-Sharp
rgb = image.select('B4', 'B3', 'B2')
pan = image.select('B8')

# แปลงเป็น HSV, สลับในแถบ PAN และแปลงกลับเป็น RGB
huesat = rgb.rgbToHsv().select('hue', 'saturation')
upres = ee.Image.cat([huesat, pan]).hsvToRgb()

In [4]:
# สร้างแผนที่ (ลองเปลี่ยนเป็น Shanghai Disneyland)
Map = geemap.Map(center=[31.1434, 121.6570], zoom=14)

# แสดงเลเยอร์ ก่อนและหลัง โดยใช้พารามิเตอร์ vis เดียวกัน
Map.addLayer(rgb, {'max': 0.28}, 'Original')
Map.addLayer(pan, {'max': 0.28}, 'Pan')
Map.addLayer(upres, {'max': 0.28}, 'Pansharpened')
Map


Map(center=[31.1434, 121.657], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

คำถาม
ข้อเพื่อทดสอบความเข้าใจของนักศึกษาเกี่ยวกับเทคนิคการทำ Pan-sharpening ด้วยภาพ Landsat-9 จงตอบคำถามต่อไปนี้

1. อะไรคือเป้าหมายหลักของการปรับความคมชัดของภาพในการสำรวจระยะไกล โดยเฉพาะเมื่อใช้ภาพ Landsat-9 อธิบายว่าเหตุใดจึงมีความสำคัญในการประมวลผลภาพ

**ตอบ**
เป้าหมายหลักและความสำคัญของ Pan-sharpening ใน Landsat-9 คือการสร้างข้อมูลภาพใหม่ที่รวมเอาข้อดีของเซนเซอร์สองระบบเข้าด้วยกัน คือการนำ ความละเอียดเชิงพื้นที่สูง (High Spatial Resolution) จากแบนด์ Panchromatic (Band 8 ของ Landsat-9 ซึ่งมีความละเอียด 15 เมตร)มารวมกับ                                                                                                                                                                                                                                                                                   ข้อมูลสีหรือรายละเอียดเชิงคลื่นความถี่ (Spectral Resolution) จากแบนด์ Multispectral (Band 1-7 ซึ่งมีความละเอียด 30 เมตร) โดยความสำคัญในการประมวลผลคือเพื่อเพิ่มขีดความสามารถในการแปลภาพ คือ ช่วยให้การแปลภาพด้วยสายตา (Visual Interpretation) แม่นยำขึ้น เห็นรายละเอียดวัตถุขนาดเล็ก เช่น อาคารหรือถนนได้ชัดเจน โดยที่ยังรักษาข้อมูลสีซึ่งจำเป็นต่อการจำแนกประเภทข้อมูล (Classification) ไว้ได้


2. อธิบายขั้นตอนสำคัญที่เกี่ยวข้องกับการปรับความคมชัดด้วนเทคนิค Pan-sharpening ตั้งแต่การรับข้อมูลไปจนถึงการสร้างภาพที่ปรับความคมชัด เทคนิค Pan-sharpening มี กระบวนการสุ่มตัวอย่างใหม่ช่วยจัดแนวภาพหลายสเปกตรัมและภาพแพนโครมาติกอย่างไร

**ตอบ**
ขั้นตอนหลักในการทำ Pan-sharpening:

1.) Data Acquisition & Pre-processing รับข้อมูล Landsat-9 และตรวจสอบความถูกต้องเชิงตำแหน่ง (Geometric Correction) ให้แน่ใจว่าทั้งภาพ MS และ PAN วางซ้อนทับกันพอดี

2.) Co-registration ปรับแก้ตำแหน่งของภาพทั้งสองชุดให้ตรงกันในระดับพิกเซล (Alignment)

3.) Resampling ขยายขนาดภาพ Multispectral (30m) ให้มีขนาดพิกเซลเท่ากับภาพ Panchromatic (15m)

4.) Transformation/Fusion: ใช้เทคนิคทางคณิตศาสตร์
(เช่น IHS, Brovey หรือ PCA) เพื่อสกัดค่าความสว่างจากภาพ PAN ไปใส่ในภาพ MS

5.) Inverse Transformation: แปลงข้อมูลกลับมาเป็นภาพสีที่มีรายละเอียดความคมชัดสูง

ในส่วนของกระบวนการสุ่มตัวอย่างใหม่ (Resampling) มีความสำคัญมากในการ "จัดแนว (Alignment)" เพราะเครื่องมือ Pan-sharpening ต้องการให้โครงสร้างตารางพิกเซลของทั้งสองภาพมีขนาดเท่ากันก่อนจะนำมาคำนวณรวมกันโดยระบบจะสร้างตารางพิกเซลใหม่ที่มีขนาด 15x15 เมตร (เท่ากับ PAN) ขึ้นมาสำหรับภาพ Multispectral และการประมาณค่าใช้วิธีอย่างเช่น Bilinear Interpolation หรือ Cubic Convolution เพื่อคำนวณค่าสีใหม่ลงในตารางที่เล็กลงนั้น ทำให้ผลลัพธ์ที่ได้พิกเซลของสี (MS) และพิกเซลของรายละเอียด (PAN) ซ้อนทับกันแบบ 1:1 ส่งผลให้เมื่อรวมภาพแล้ว สีจะไม่เหลื่อมออกจากขอบวัตถุ (No color bleeding)

นางสาว ณัฏฐณิชา ปลอดเหตุ 6506614038